<a href="https://colab.research.google.com/github/maiaufrrj/BRtrading/blob/main/BD_Binance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

referência binance
https://www.youtube.com/watch?v=rc_Y6rdBqXM&t=79s

conectar google colab com sqlite3: https://www.youtube.com/watch?v=3696yt6JWHM

In [ ]:
#! pip install python-binance

In [ ]:
import pandas as pd
import sqlalchemy
from binance.client import Client
from binance import BinanceSocketManager

In [ ]:
from google.colab import drive
import warnings
warnings.filterwarnings("ignore")
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
api_key = 'XXXXXXXXXXXXXXXXXXXXXXXX'
api_secret = 'XXXXXXXXXXXXXXXXXXXXXXXX'

In [ ]:
client = Client(api_key, api_secret)

In [ ]:
bsm = BinanceSocketManager(client)

In [ ]:
socket = bsm.trade_socket('BTCUSDT')

In [ ]:
engine = sqlalchemy.create_engine('sqlite:////content/drive/MyDrive/Colab Notebooks/BTCUSDTstream.db')

In [ ]:
def createframe(msg):
  df = pd.DataFrame([msg])
  df = df.loc[:,['s','E','p']]
  df.columns = ['symbol','Time','Price']
  df.Price = df.Price.astype(float)
  df.Time = pd.to_datetime(df.Time, unit='ms')
  return df

In [ ]:
msg

<coroutine object ReconnectingWebsocket.recv at 0x7ff06daf84d0>

In [ ]:
while True:
  await socket.__aenter__()
  msg = await socket.recv()
  frame = createframe(msg)
  frame.to_sql('BTCUSDT', engine, if_exists='append', index=False)
  print(frame)

In [ ]:
#import time
#from datetime import datetime

#while True:
#  time.sleep(1) # Sleep for 1 seconds  
#  now = datetime.now()
#  print(now)

In [ ]:
df = pd.read_sql('BTCUSDT', engine)

In [ ]:
df.head()

,symbol,Time,Price
0,BTCUSDT,2021-11-03 00:35:45.876,63037.42
1,BTCUSDT,2021-11-03 00:35:45.876,63037.42
2,BTCUSDT,2021-11-03 00:35:45.919,63037.42
3,BTCUSDT,2021-11-03 00:35:45.999,63037.42
4,BTCUSDT,2021-11-03 00:35:46.006,63037.42


In [ ]:
import plotly.express as px
fig = px.line(df, x=df.Time, y='Price')
fig.show()

In [ ]:
#seguidor de tendência
#se o ticker subir x% ==> comprar
#sair se o lucro estiver acima de 0.15% ou se a perda for de -0.15%

In [ ]:
def estrategia1(entrada, lookback, qty, open_position=False, stop_position):
  while True:
    df = pd.read_sql('BTCUSDT', engine)
    lookbackperiod = df.iloc[-lookback:]
    retorno_acumulado = (lookbackperiod.Price.pct_change()+1).cumprod()-1
    if not open_position:
      if retorno_acumulado[retorno_acumulado.last_valid_index()] > entrada:
        order=client.create_order(symbol='BTCUSDT',
                                  side='BUY',
                                  type='MARKET',
                                  quantity=qty)
        print(order)
        open_position = True
        break
  if open_position:
    while True:
      df = pd.read_sql('BTCUSDT', engine)
      sincebuy = df.loc[df.Time > pd.to_datetime(order['transactTime', unit='ms'])]
      if len(sincebuy) > 1:
        sincebuyret = sincebuy.Price.pct-change()+1).cumprod()-1
        last_entry = sincebuyret[sincebuyret.last_valid_index()]
        if sincebuyret> stop_position or last_entry < -stop_position:
          order=client.create_order(symbol='BTCUSDT',
                                  side='SELL',
                                  type='MARKET',
                                  quantity=qty)
          
          print(order)
          break

In [ ]:
estrategia1(entrada=0.001,lookback=60, qty=0.001, stop_position=0.0015)